# Explaination of Lab

## 00-a Clone course repository
- First of all clone the LAB repository in your own github 

## 00-b Gitignore
- After creating the lab repositry in your github create the gitignore file and put following to ignore them during your lab project
- .wal
- .DS Store
- .dbeaver
- .project

## 00-c Changing folder name

- To perform this lab , we have alot of data in different schemas and we want to change the name of those schemas.
- For example from Datum 2024-08-13_2024-09-10 AIgineer and those schemas we have in lab in backend folder - raw data.
- We want to keep the schema name like Datum and rest remove it and copy them in clean data folder. 

### To perform this action we did some coding.

- Frist to all mention the path for raw data folder with the help of Pathlib and same for clean data folder 
- Create the clean folder so copy the renamed schemas  and this copying the schemas is happended with help of #copytree
- With the help of below block of code , change the raw schema into new schema name. 
```python
for folder in raw_data_path.iterdir():
    new_name = folder.name.split()[0]
    copytree(folder, cleaned_data_path / new_name)
```


# 00-d Ingesting the CSV file in database with help of Python

- We have two CSV files in data folder and find where is the CSV file is with help of DATA_PATH.
- With the hep of setup_schema funtions set a schema with the name of supplier_invoice.
- Clean up the file name , create a table for those invoices and import the CSV data into the data.  